In [1]:
import pandas as pd
import requests
import os
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
if not os.path.exists('./nfl_data/weekly/cbs_weekly'):
    os.mkdir('./nfl_data/weekly/cbs_weekly')

#### Scrapes CBS Weekly for all positions & all weeks
Saves files to `cbs_weekly` directory

In [9]:
positions = ['QB', 'WR', 'RB', 'TE', 'K']
weeks = [19, 20, 21, 23] # 22 was pro bowl

file_list = []

for position in positions:
    for week in weeks:
        url = f'https://www.cbssports.com/nfl/stats/leaders/live/{position}/{week}/'
        print(url)
        res = requests.get(url)
        print(res)
        output = open(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls', 'wb')
        output.write(res.content)
        file_list.append(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        output.close()
        data = pd.read_html(f'./nfl_data/weekly/cbs_weekly/week-{week}_{position}.xls')
        df = data[0]
        print((position, week), df.shape)

https://www.cbssports.com/nfl/stats/leaders/live/QB/19/
<Response [200]>
('QB', 19) (13, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/20/
<Response [200]>
('QB', 20) (8, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/21/
<Response [200]>
('QB', 21) (3, 11)
https://www.cbssports.com/nfl/stats/leaders/live/QB/23/
<Response [200]>
('QB', 23) (2, 11)
https://www.cbssports.com/nfl/stats/leaders/live/WR/19/
<Response [200]>
('WR', 19) (56, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/20/
<Response [200]>
('WR', 20) (39, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/21/
<Response [200]>
('WR', 21) (14, 10)
https://www.cbssports.com/nfl/stats/leaders/live/WR/23/
<Response [200]>
('WR', 23) (10, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/19/
<Response [200]>
('RB', 19) (35, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/20/
<Response [200]>
('RB', 20) (19, 10)
https://www.cbssports.com/nfl/stats/leaders/live/RB/21/
<Response [200]>


In [10]:
file_list

['./nfl_data/weekly/cbs_weekly/week-19_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-20_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-21_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-23_QB.xls',
 './nfl_data/weekly/cbs_weekly/week-19_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-20_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-21_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-23_WR.xls',
 './nfl_data/weekly/cbs_weekly/week-19_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-20_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-21_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-23_RB.xls',
 './nfl_data/weekly/cbs_weekly/week-19_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-20_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-21_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-23_TE.xls',
 './nfl_data/weekly/cbs_weekly/week-19_K.xls',
 './nfl_data/weekly/cbs_weekly/week-20_K.xls',
 './nfl_data/weekly/cbs_weekly/week-21_K.xls',
 './nfl_data/weekly/cbs_weekly/week-23_K.xls']

In [11]:
# get df started with the 1st file 'week-19_TE.xls'.  This way we have something to merge to.

df_te = pd.read_html('./nfl_data/weekly/cbs_weekly/week-19_TE.xls')
df_te = pd.DataFrame(df_te[0])
df_te = df_te.droplevel(0, axis=1)
df_te['Week'] = re.search(r'(?<=\-)\s*(..)', file_list[0])[0]
df_te['Pos'] = [i.split()[-1] for i in df_te['Player  Player on team']]
df_te.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
df_te['Player'] = df_te['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
df_te

,Player,Game,FPTS Fantasy Points,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
0,Travis Kelce,KC 42 - PIT 21,22,5,108,7,1,—,—,—,19,TE
1,Dawson Knox,BUF 47 - NE 17,20,5,89,5,2,—,—,—,19,TE
2,C.J. Uzomah,CIN 26 - LV 19,12,6,64,6,1,—,—,—,19,TE
3,Dallas Goedert,TB 31 - PHI 15,9,6,92,12,0,—,—,—,19,TE
4,Rob Gronkowski,TB 31 - PHI 15,9,5,31,6,1,—,—,—,19,TE
5,Dalton Schultz,SF 23 - DAL 17,8,7,89,8,0,—,—,—,19,TE
6,Darren Waller,CIN 26 - LV 19,7,7,76,12,0,—,—,—,19,TE
7,Tyler Higbee,LAR 34 - ARI 11,4,3,46,4,0,—,—,—,19,TE
8,Hunter Henry,BUF 47 - NE 17,3,1,30,4,0,—,—,—,19,TE
9,Zach Gentry,KC 42 - PIT 21,3,4,33,4,0,—,—,—,19,TE


In [12]:
df_te.shape

(22, 12)

In [13]:
for file in file_list:
    if 'TE' in file:
        print(file)
        data = pd.read_html(file)
        df = data[0]
        df = df.droplevel(0, axis=1)
        df['Week'] = re.search(r'(?<=\-)\s*(..)', file)[0]
        df['Pos'] = [i.split()[-1] for i in df['Player  Player on team']]
        df.rename(columns={'Player  Player on team': 'Player'}, inplace=True)
        df['Player'] = df['Player'].map(lambda x: x.split()[3] + ' ' + x.split()[4])
        dfs = [df_te, df]
        df_te = pd.concat([df_te.squeeze() for df_te in dfs], ignore_index=True)

print(f'The shape of the updated df is {df_te.shape}')

./nfl_data/weekly/cbs_weekly/week-19_TE.xls
./nfl_data/weekly/cbs_weekly/week-20_TE.xls
./nfl_data/weekly/cbs_weekly/week-21_TE.xls
./nfl_data/weekly/cbs_weekly/week-23_TE.xls
The shape of the updated df is (66, 12)


In [14]:
df_te.sort_values(by=['Player', 'Week'])

,Player,Game,FPTS Fantasy Points,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
59,Anthony Firkser,CIN 19 - TEN 16,—,0,0,1,0,—,—,—,20,TE
20,Blake Bell,KC 42 - PIT 21,—,1,3,1,0,—,—,—,19,TE
42,Blake Bell,KC 42 - PIT 21,—,1,3,1,0,—,—,—,19,TE
18,Blake Jarwin,SF 23 - DAL 17,—,0,0,1,0,—,—,—,19,TE
40,Blake Jarwin,SF 23 - DAL 17,—,0,0,1,0,—,—,—,19,TE
64,Brycen Hopkins,LAR 23 - CIN 20,8,4,47,4,0,—,—,—,23,TE
2,C.J. Uzomah,CIN 26 - LV 19,12,6,64,6,1,—,—,—,19,TE
24,C.J. Uzomah,CIN 26 - LV 19,12,6,64,6,1,—,—,—,19,TE
45,C.J. Uzomah,CIN 19 - TEN 16,14,7,71,8,0,—,—,—,20,TE
63,C.J. Uzomah,CIN 27 - KC 24,—,0,0,2,0,—,—,—,21,TE


In [15]:
df_te.columns

Index(['Player', 'Game', 'FPTS  Fantasy Points', 'REC  Receptions',
       'YDS  Receiving Yards', 'TGT  Targets', 'TD  Receiving Touchdowns',
       'ATT  Rushing Attempts', 'YDS  Rushing Yards', 'TD  Rushing Touchdowns',
       'Week', 'Pos'],
      dtype='object')

In [16]:
df_te = df_te.replace('—', 0) # get rid of the dashes, impute with 0
df_te.head()

,Player,Game,FPTS Fantasy Points,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,Week,Pos
0,Travis Kelce,KC 42 - PIT 21,22,5,108,7,1,0,0,0,19,TE
1,Dawson Knox,BUF 47 - NE 17,20,5,89,5,2,0,0,0,19,TE
2,C.J. Uzomah,CIN 26 - LV 19,12,6,64,6,1,0,0,0,19,TE
3,Dallas Goedert,TB 31 - PHI 15,9,6,92,12,0,0,0,0,19,TE
4,Rob Gronkowski,TB 31 - PHI 15,9,5,31,6,1,0,0,0,19,TE


In [17]:
# Attempt to convert all columns to int, those that fail are stored in list
numcols_to_change = df_te.columns
numcols_to_change2 = []
for col in numcols_to_change:
    try:
        df_te[col] = df_te[col].astype(int)
        print('success!')
    except:
        numcols_to_change2.append(col)
        print(f'need to clean column: {col}')

need to clean column: Player
need to clean column: Game
success!
success!
success!
success!
success!
success!
success!
success!
success!
need to clean column: Pos


In [18]:
max_week = df_te['Week'].max()
df_te.to_csv(f'./nfl_data/weekly/cbs_weekly/tight_ends_through_week-{max_week}.csv', index=0)